## ## Tuning Hyperparameters with Kears Tuner 2

- Task: Find best Number of Filters in ConvLayers and best Learning Rate for SGD Optimizer
- Dataset: Cropped and Resized Fundus Images
- Original Colorscheme

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from tensorflow.keras.callbacks import EarlyStopping

In [1]:
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Anomaly Detection/train_binary.csv')

In [6]:
df.normal = df.normal.astype('str')

In [7]:
df.normal.value_counts()

1    3150
0    2910
Name: normal, dtype: int64

In [8]:
# As data is ordered in dataframe after augmentation, shuffle it
from sklearn.utils import shuffle 
df = shuffle(df)

In [9]:
# Directory
indir = r"D:\data\Projects\notebooks\RetinaAI\Anomaly Detection\train"

In [10]:
datagen = ImageDataGenerator(rescale=1./255., validation_split = 0.25)

In [11]:
train_gen = datagen.flow_from_dataframe(dataframe = df, 
                                            directory = indir, 
                                            x_col = "filename", 
                                            y_col = 'normal',
                                            batch_size = 64, 
                                            seed = 2, 
                                            shuffle = True, 
                                            class_mode = "binary", 
                                            target_size = (300,300),
                                            subset='training')

Found 4545 validated image filenames belonging to 2 classes.


In [12]:
val_gen = datagen.flow_from_dataframe(dataframe = df, 
                                            directory = indir, 
                                            x_col = "filename", 
                                            y_col = 'normal',
                                            batch_size = 64, 
                                            seed = 2, 
                                            shuffle = True, 
                                            class_mode = "binary", 
                                            target_size = (300,300),
                                            subset='validation')

Found 1515 validated image filenames belonging to 2 classes.


Tune Hyperparameters

In [16]:
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [17]:
LOG_DIR = 'log_dir'

In [18]:
def build_model(hp):
    model = Sequential()

    model.add(Conv2D(hp.Int('conv_1', min_value=32, max_value=128, step=32), 
                     (3, 3), kernel_initializer='he_uniform', padding='same', activation='relu',  input_shape=(300,300,3)))
    model.add(Conv2D(hp.Int('conv_2', min_value=32, max_value=128, step=32), 
                     (3, 3), kernel_initializer='he_uniform', padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(hp.Int('conv_3', min_value=64, max_value=256, step=64), 
                     (3, 3), kernel_initializer='he_uniform', padding='same', activation='relu'))
    model.add(Conv2D(hp.Int('conv_4', min_value=64, max_value=256, step=64), 
                     (3, 3), kernel_initializer='he_uniform', padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(hp.Int('conv_5', min_value=64, max_value=256, step=64), 
                     (3, 3), kernel_initializer='he_uniform', padding='same', activation='relu')))
    model.add(Conv2D(hp.Int('conv_6', min_value=64, max_value=256, step=64), 
                     (3, 3), kernel_initializer='he_uniform', padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Flatten())
    model.add(Dense(hp.Int('units', min_value=64, max_value=256, step=64), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    
    lr = hp.Choice("learning_rate", values=[1e-1, 1e-2, 1e-3])
    opt = SGD(learning_rate=lr, momentum=0.9)
    
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    
    return(model)

In [19]:
tuner = Hyperband(build_model,
                     objective="val_accuracy",
                     max_epochs=20,
                     factor=3,seed=2,
                     directory=config.OUTPUT_PATH,
                     project_name=args["tunes"])

In [20]:
step_size_train = train_gen.n//train_gen.batch_size
step_size_val = val_gen.n//val_gen.batch_size

In [ ]:
es = EarlyStopping(monitor='val_acc', patience = 5)

In [21]:
tuner.search(x=train_gen, 
             validation_data=val_gen, 
             steps_per_epoch=step_size_train, 
             validation_steps=step_size_val, 
             epochs=1, 
             verbose=2
             )

Trial 1 Complete [00h 32m 56s]
val_accuracy: 0.9245923757553101

Best val_accuracy So Far: 0.9245923757553101
Total elapsed time: 00h 32m 56s
INFO:tensorflow:Oracle triggered exit


In [24]:
print(tuner.results_summary())

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
input_units: 64
Score: 0.9245923757553101
None


In [23]:
tuner.get_best_models()[0].summary()
#tuner.get_best_models()[0].predict()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 296, 296, 32)      18464     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 146, 146, 64)      18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 144, 144, 32)      18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 72, 72, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 70, 70, 64)        1